##  Aggregations



### Step 1: Initialize PySpark Session


In [9]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, count, when, lit , avg, coalesce , struct,array , explode, create_map,approx_count_distinct,sumDistinct, sum, mean
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, FloatType

# Create a Spark session
spark = SparkSession.builder.appName("day4").getOrCreate()


In [3]:
# Load the Chipotle dataset into a Spark DataFrame
data_path = "./US_Crime_Rates_1960_2014.csv"  # Replace with the actual path
US_df = spark.read.csv(data_path, header=True, inferSchema=True)

# Load the Chipotle dataset into a Spark DataFrame
data_path = "./titanic.csv"  # Replace with the actual path
titanic_df = spark.read.csv(data_path, header=True, inferSchema=True)

In [12]:
#creating a temporary view for the "US_df" DataFrame
US_df.createOrReplaceTempView("us_data")

In [4]:
US_df.printSchema()

root
 |-- Year: integer (nullable = true)
 |-- Population: integer (nullable = true)
 |-- Total: integer (nullable = true)
 |-- Violent: integer (nullable = true)
 |-- Property: integer (nullable = true)
 |-- Murder: integer (nullable = true)
 |-- Forcible_Rape: integer (nullable = true)
 |-- Robbery: integer (nullable = true)
 |-- Aggravated_assault: integer (nullable = true)
 |-- Burglary: integer (nullable = true)
 |-- Larceny_Theft: integer (nullable = true)
 |-- Vehicle_Theft: integer (nullable = true)



In [5]:
titanic_df.printSchema()

root
 |-- PassengerId: integer (nullable = true)
 |-- Survived: integer (nullable = true)
 |-- Pclass: integer (nullable = true)
 |-- Name: string (nullable = true)
 |-- Sex: string (nullable = true)
 |-- Age: double (nullable = true)
 |-- SibSp: integer (nullable = true)
 |-- Parch: integer (nullable = true)
 |-- Ticket: string (nullable = true)
 |-- Fare: double (nullable = true)
 |-- Cabin: string (nullable = true)
 |-- Embarked: string (nullable = true)



### count

Question: How many records are there in the US_Crime_Rates_1960_2014_df DataFrame?

In [15]:
#counting the number of records in the "US_df" DataFrame
record_count = spark.sql("SELECT COUNT(*) AS record_count FROM us_data")
result = record_count.first()

# Extract the count from the result
count = result['record_count']
print("Number of records:", count)

Number of records: 55


### countDistinct
Question: How many distinct years are present in the US_Crime_Rates_1960_2014_df DataFrame?
Answer:

In [20]:
#counting the number of distinct years in the dataframe
distinct_years_count = spark.sql("SELECT COUNT(DISTINCT Year) AS distinct_years_count FROM us_data")
result = distinct_years_count.first()

#extracting the count from the result
count = result['distinct_years_count']
print("Number of distinct years:", count)

Number of distinct years: 55


### approx_count_distinct

Question: Estimate the approximate number of distinct values in the "Total" column of the US_Crime_Rates_1960_2014_df DataFrame.

In [23]:
#estimating the approximate number of distinct values in the "Total" column
approx_distinct_count = spark.sql("SELECT approx_count_distinct(Total) AS approx_distinct_total FROM us_data")
result = approx_distinct_count.first()

#extracting
#  the estimated count from the result
approx_count = result['approx_distinct_total']
print("Approximate distinct values in 'Total' column:", approx_count)

Approximate distinct values in 'Total' column: 55


###  first and last

Question: Find the first and last year in the US_Crime_Rates_1960_2014_df DataFrame.

In [27]:
#finding the first and last year in the dataframe
year_stats = spark.sql("""
    SELECT MIN(Year) AS first_year, MAX(Year) AS last_year
    FROM us_data
""")
result = year_stats.first()

#extracting the first and last year from the result
first_year = result['first_year']
last_year = result['last_year']

print("First year:", first_year)
print("Last year:", last_year)

First year: 1960
Last year: 2014


### min and max

Question: Find the minimum and maximum population values in the US_Crime_Rates_1960_2014_df DataFrame.

In [26]:
#finding the minimum and maximum population values in the dataframe
population_stats = spark.sql("""
    SELECT MIN(Population) AS min_population, MAX(Population) AS max_population
    FROM us_data
""")
result = population_stats.first()

#extracting the minimum and maximum population values from the result
min_population = result['min_population']
max_population = result['max_population']

print("Minimum population:", min_population)
print("Maximum population:", max_population)

Minimum population: 179323175
Maximum population: 318857056


### sumDistinct

Question: Calculate the sum of distinct "Property" values for each year in the US_Crime_Rates_1960_2014_df DataFrame.

In [30]:
#calculating the sum of distinct "Property" values for each year

distinct_property_sum = spark.sql("""
    SELECT Year, SUM(DISTINCT Property) AS Sumdistinctproperty
    FROM us_data
    GROUP BY Year
    ORDER BY Year
""")
#showing the values
distinct_property_sum.show()

+----+-------------------+
|Year|Sumdistinctproperty|
+----+-------------------+
|1960|            3095700|
|1961|            3198600|
|1962|            3450700|
|1963|            3792500|
|1964|            4200400|
|1965|            4352000|
|1966|            4793300|
|1967|            5403500|
|1968|            6125200|
|1969|            6749000|
|1970|            7359200|
|1971|            7771700|
|1972|            7413900|
|1973|            7842200|
|1974|            9278700|
|1975|           10252700|
|1976|           10345500|
|1977|            9955000|
|1978|           10123400|
|1979|           11041500|
+----+-------------------+
only showing top 20 rows



### avg

Question: Calculate the average "Murder" rate for the entire dataset in the US_Crime_Rates_1960_2014_df DataFrame.
Answer:

In [32]:
#calculating the average "Murder" rate for the entire dataset
average_murder_rate = spark.sql("SELECT AVG(Murder) AS avg_murder_rate FROM us_data")
result = average_murder_rate.first()

#extracting the average "Murder" rate from the result
avg_murder_rate = result['avg_murder_rate']

print("Average Murder rate:", avg_murder_rate)

Average Murder rate: 17317.236363636363


### Aggregating to Complex Types

Question: Calculate the total sum of "Violent" and "Property" crimes for each year in the US_Crime_Rates_1960_2014_df DataFrame. Store the results in a struct type column.

In [33]:
#calculating the total sum of "Violent" and "Property" crimes for each year and create a struct column
result_with_struct = spark.sql("""
    SELECT Year, 
           struct(SUM(Violent) AS TotalViolent, SUM(Property) AS TotalProperty) AS CrimeTotals
    FROM us_data
    GROUP BY Year
    ORDER BY Year
""")
result_with_struct.show(truncate=False)

+----+-------------------+
|Year|CrimeTotals        |
+----+-------------------+
|1960|{288460, 3095700}  |
|1961|{289390, 3198600}  |
|1962|{301510, 3450700}  |
|1963|{316970, 3792500}  |
|1964|{364220, 4200400}  |
|1965|{387390, 4352000}  |
|1966|{430180, 4793300}  |
|1967|{499930, 5403500}  |
|1968|{595010, 6125200}  |
|1969|{661870, 6749000}  |
|1970|{738820, 7359200}  |
|1971|{816500, 7771700}  |
|1972|{834900, 7413900}  |
|1973|{875910, 7842200}  |
|1974|{974720, 9278700}  |
|1975|{1039710, 10252700}|
|1976|{1004210, 10345500}|
|1977|{1029580, 9955000} |
|1978|{1085550, 10123400}|
|1979|{1208030, 11041500}|
+----+-------------------+
only showing top 20 rows



### Grouping

Question: In the given US_Crime_Rates_1960_2014_df DataFrame, you are tasked with finding the average of all crimes combined for each year. Calculate the sum of all crime categories (Violent, Property, Murder, Forcible_Rape, Robbery, Aggravated_assault, Burglary, Larceny_Theft, Vehicle_Theft) for each year and then determine the average of these combined crime sums. Provide the result as the average of all crimes across the entire dataset.

In [35]:
# Calculate the sum of all crime categories for each year
crime_sums = spark.sql("""
    SELECT Year, 
           SUM(Violent + Property + Murder + Forcible_Rape + Robbery + Aggravated_assault + Burglary + Larceny_Theft + Vehicle_Theft) AS TotalCrime
    FROM us_data
    GROUP BY Year
    ORDER BY Year
""")

# Calculate the average of all crimes across the entire dataset
average_all_crimes = spark.sql("""
    SELECT AVG(TotalCrime) AS AvgAllCrimes
    FROM (
        SELECT SUM(Violent + Property + Murder + Forcible_Rape + Robbery + Aggravated_assault + Burglary + Larceny_Theft + Vehicle_Theft) AS TotalCrime
        FROM us_data
    )
""")
                               
average_all_crimes.show()

+-------------+
| AvgAllCrimes|
+-------------+
|1.166085038E9|
+-------------+



### Grouping

Question: In the given US_Crime_Rates_1960_2014_df DataFrame, you are tasked with finding the average of all crimes combined for each year. Calculate the sum of all crime categories (Violent, Property, Murder, Forcible_Rape, Robbery, Aggravated_assault, Burglary, Larceny_Theft, Vehicle_Theft) for each year and then determine the average of these combined crime sums. Provide the result as the average of all crimes across the entire dataset.

In [39]:
#calculating the total sum of all crime categories for each year
total_sum = spark.sql(""" 
                      SELECT *,
                            (Violent + Property + Murder + Forcible_rape + Robbery +
                            Aggravated_assault + Burglary + Larceny_Theft + Vehicle_Theft) AS Total_crime_sum
                     FROM us_data
                    """)

# total_sum.show()


avegare_crime = total_sum.agg(avg('total_crime_sum'))
print(f"Average of all crime: {avegare_crime.collect()[0][0]}")

total_sum.select('year','total_crime_sum').show()
# avegare_crime.show()
# average_crime = total_sum.groupBy('year').agg(avg('total_crime_sum').alias('average_crime'))
# average_crime.show()

Average of all crime: 21201546.145454545
+----+---------------+
|year|total_crime_sum|
+----+---------------+
|1960|        6768320|
|1961|        6975980|
|1962|        7504420|
|1963|        8218940|
|1964|        9129240|
|1965|        9478780|
|1966|       10446960|
|1967|       11806860|
|1968|       13440420|
|1969|       14821740|
|1970|       16196040|
|1971|       17176400|
|1972|       16497600|
|1973|       17436220|
|1974|       20506940|
|1975|       22584730|
|1976|       22699410|
|1977|       21969060|
|1978|       22417910|
|1979|       24499060|
+----+---------------+
only showing top 20 rows



### Window Functions

Question: Calculate the cumulative sum of "Property" values over the years using a window function in the US_Crime_Rates_1960_2014_df DataFrame.

In [44]:
# Calculate the cumulative sum of "Property" values with a partition specification
cumulative_property_sum = spark.sql("""
    SELECT *,
           SUM(Property) OVER (PARTITION BY Year ORDER BY Year) AS Cumulative_Property_Sum
    FROM us_data
    ORDER BY Year
""")

# Show the result
cumulative_property_sum.show()

+----+----------+--------+-------+--------+------+-------------+-------+------------------+--------+-------------+-------------+-----------------------+
|Year|Population|   Total|Violent|Property|Murder|Forcible_Rape|Robbery|Aggravated_assault|Burglary|Larceny_Theft|Vehicle_Theft|Cumulative_Property_Sum|
+----+----------+--------+-------+--------+------+-------------+-------+------------------+--------+-------------+-------------+-----------------------+
|1960| 179323175| 3384200| 288460| 3095700|  9110|        17190| 107840|            154320|  912100|      1855400|       328200|                3095700|
|1961| 182992000| 3488000| 289390| 3198600|  8740|        17220| 106670|            156760|  949600|      1913000|       336000|                3198600|
|1962| 185771000| 3752200| 301510| 3450700|  8530|        17550| 110860|            164570|  994300|      2089600|       366800|                3450700|
|1963| 188483000| 4109500| 316970| 3792500|  8640|        17650| 116470|          

### Pivot
Question: You are working with a DataFrame named US_Crime_Rates_1960_2014_df that contains crime data for different crime types over the years. 